In [2]:
import segmentation_models as sm
import numpy as np
import sys, os
from tensorflow.keras.utils import normalize
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

Segmentation Models: using `keras` framework.


In [3]:
sys.path.insert(1, 'E:/Research/course research projects/data-driven software engineering/gui-element-detection/src')
from constants import VINS_MERGED_YOLO_SPLITS_IMAGES as imgs_path
from constants import VINS_MERGED_YOLO_SPLITS_LABELS as labels_path
from util.data_utils import LazyDataLoaderV2, Splits
# from ..util.data_utils import LazyDataLoader, Splits
# from ..constants import VINS_MERGED_YOLO_SPLITS_IMAGES as imgs_path
# from ..constants import VINS_MERGED_YOLO_SPLITS_LABELS as labels_path

In [4]:
batch_size = 12
classes = 12 + 1

In [5]:
def get_data_loader(split):
    return LazyDataLoaderV2(split=split, images_path=imgs_path, labels_path=labels_path, batch_size=batch_size,
                          n_classes=classes, resize=True, shape=(416, 416), normalize=True)

In [6]:
train_loader = get_data_loader(Splits.TRAIN)
validation_loader = get_data_loader(Splits.VALIDATION)
test_loader = get_data_loader(Splits.TEST)

In [7]:


activation = 'softmax'
learning_rate = 0.0001
opt = Adam(learning_rate)
# dice_loss = sm.losses.DiceLoss()  # makes sense as the end result is highly imbalanced
# class_weights=tf.convert_to_tensor(np.array([0.25, 0.25, 0.25, 0.25]))
# focal_loss = sm.losses.CategoricalFocalLoss()  # why not Categorical CE ?
# total_loss = dice_loss + (focal_loss * 1)
total_loss = sm.losses.CategoricalCELoss()
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.IOUScore(threshold=0.9,name='iou score @ 0.9'), sm.metrics.FScore(threshold=0.5),
           sm.metrics.Precision(), sm.metrics.Recall()]  # this is all they have :|

In [8]:
RESNET_BACKBONE = 'resnet34'
preprocess_inputs = sm.get_preprocessing(RESNET_BACKBONE)

train_loader = preprocess_inputs(train_loader)
test_loader = preprocess_inputs(test_loader)
validation_loader = preprocess_inputs(validation_loader)

In [9]:
model1 = sm.Unet(RESNET_BACKBONE, encoder_weights='imagenet', classes=13, activation=activation)
model1.compile(opt, total_loss, metrics=metrics)

In [10]:
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 bn_data (BatchNormalization)   (None, None, None,   9           ['data[0][0]']                   
                                3)                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, None, None,   0          ['bn_data[0][0]']                
                                3)                                                          

In [10]:
history = model1.fit(train_loader,
                      epochs=100,
                      verbose=1,
                      validation_data=validation_loader)

Epoch 1/100
303/303 [==============================] - 176s 539ms/step - loss: 0.1144 - iou_score: 0.3466 - iou score @ 0.9: 0.2492 - f1-score: 0.3888 - precision: 0.1595 - recall: 0.4265 - val_loss: 0.1412 - val_iou_score: 0.2734 - val_iou score @ 0.9: 0.2328 - val_f1-score: 0.2855 - val_precision: 0.0845 - val_recall: 0.3086
Epoch 2/100
303/303 [==============================] - 165s 542ms/step - loss: 0.0573 - iou_score: 0.4410 - iou score @ 0.9: 0.2195 - f1-score: 0.4959 - precision: 0.2867 - recall: 0.4973 - val_loss: 0.1033 - val_iou_score: 0.3117 - val_iou score @ 0.9: 0.1897 - val_f1-score: 0.3640 - val_precision: 0.2067 - val_recall: 0.3577
Epoch 3/100
303/303 [==============================] - 167s 549ms/step - loss: 0.0408 - iou_score: 0.5138 - iou score @ 0.9: 0.2955 - f1-score: 0.5646 - precision: 0.3693 - recall: 0.5761 - val_loss: 0.0447 - val_iou_score: 0.5354 - val_iou score @ 0.9: 0.3264 - val_f1-score: 0.5911 - val_precision: 0.3795 - val_recall: 0.5689
Epoch 4/100
3

In [11]:
model1.save('res34bb_UNET_60epochs_new.hdf5')

In [15]:
model1.compute_metrics()

In [1]:
print("sds")

sds


In [13]:
m = load_model('res34bb_UNET_100epochs.hdf5',compile=False)

In [16]:
m

TypeError: 'NoneType' object is not subscriptable